In [1]:
import pandas as pd
import re
import numpy as np
import preprocessor as p

In [2]:
df_reg = pd.read_csv('/mnt/39b89ea8-8334-43c3-b50b-918be75d4569/Dropbox/Momolbox/code/libro/capitolo_bot_det/archive(1)/training.1600000.processed.noemoticon.csv',encoding='latin-1')

In [3]:
df_bot = pd.read_csv('/mnt/39b89ea8-8334-43c3-b50b-918be75d4569/Dropbox/Momolbox/code/libro/capitolo_bot_det/russian-troll-tweets/IRAhandle_tweets_1.csv')

In [4]:
sample_bot =  df_bot.content.sample(20000)
sample_regular = df_reg[df_reg.columns[5]].sample(20000)

## pulizia dei dati

In [5]:
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION)

In [6]:
sample_regular = [p.clean(s) for s in sample_regular]
sample_bot = [p.clean(s) for s in sample_bot]

In [7]:
sample_regular= [s for s in sample_regular if re.search('[a-zA-Z]', s)]
sample_bot = [s for s in sample_bot if re.search('[a-zA-Z]', s)]

In [8]:
len(sample_regular)

19948

In [9]:
len(sample_bot)

16930

In [10]:
import random


In [11]:
#selezione di un ultimo sample di 10000 tweet per parte 

sample_regular = random.sample(sample_regular, 10000)
sample_bot = random.sample(sample_bot,10000)

In [12]:
df_detect = pd.DataFrame(sample_regular,columns=['text'])

In [13]:
df_detect['flag'] ='regular'

In [14]:
df_temp = pd.DataFrame(sample_bot,columns=['text'])
df_temp['flag'] = 'bot'

In [15]:
df_detect = pd.concat([df_detect,df_temp])

In [16]:
del(df_temp)

## Classificatore

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC


In [18]:
X_train, X_test, y_train, y_test = train_test_split(df_detect['text'], df_detect['flag'], test_size=0.9, random_state=42)

# vettorizzazione
vectorizer = CountVectorizer(stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)




In [19]:
# misure di accuratezza
clf_nb = MultinomialNB()
clf_nb.fit(X_train_vec, y_train)
print('Naive Bayes accuracy:', clf_nb.score(X_test_vec, y_test))

clf_lr = LogisticRegression()
clf_lr.fit(X_train_vec, y_train)
print('Logistic Regression accuracy:', clf_lr.score(X_test_vec, y_test))

clf_svm = LinearSVC()
clf_svm.fit(X_train_vec, y_train)
print('SVM accuracy:', clf_svm.score(X_test_vec, y_test))

Naive Bayes accuracy: 0.8271666666666667
Logistic Regression accuracy: 0.8096666666666666
SVM accuracy: 0.7844444444444445


/home/novemillimetri/anaconda3/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


## Predizione su tweet nuovi 

In [20]:
tweets = df_reg[df_reg.columns[5]].sample(100)

In [21]:
tweets_clean = [p.clean(tweet) for tweet in tweets]

In [22]:
def predict_bot(text):
    
    return clf_nb.predict(vectorizer.transform([text]))[0]
    

In [23]:
results= []

for tweet in tweets: 
    results.append(predict_bot(tweet))
    

In [42]:
detected_tweets_df = pd.DataFrame(tweets)
detected_tweets_df.rename(columns={detected_tweets_df.columns[0]:'tweet'},inplace = True)

detected_tweets_df['flag'] = results

In [45]:
detected_tweets_df

,tweet,flag
43243,i love seeing my friend and my cat!!!! this is...,regular
766657,@williamyan my bad man i totally misread this ...,regular
1527865,@Habbie Try http://bit.ly/i51aO for continuou...,regular
1416874,@RonnieTomorrow No worries dude! Hope you're o...,regular
1452782,"SQL&gt; execute hello; Hello, World! PL/SQL p...",bot
...,...,...
1022277,@dduane Thanks Diane! My boyfriend and I were ...,regular
738739,Make the rain stop! I want a sunny summer,regular
303541,my Eyes feel like there bleeding,regular
517865,"just do homework , i have a exam",regular
